## Résultats

In [ ]:
import pandas as pd
import os
import numpy as np
import cv2 

In [2]:
def compute_iou(boxA, boxB):
    # Coordonnées de l'intersection
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # Calcul de l'aire d'intersection
    interArea = max(0, xB - xA) * max(0, yB - yA)

    # Aires des bounding boxes
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # IoU
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou



In [3]:
def compute_center_distance(boxA, boxB):
    centerA = ((boxA[0] + boxA[2]) / 2, (boxA[1] + boxA[3]) / 2)
    centerB = ((boxB[0] + boxB[2]) / 2, (boxB[1] + boxB[3]) / 2)
    
    # Distance Euclidienne
    return np.sqrt((centerA[0] - centerB[0]) ** 2 + (centerA[1] - centerB[1]) ** 2)



In [4]:
def compute_size_difference(boxA, boxB):
    widthA = boxA[2] - boxA[0]
    heightA = boxA[3] - boxA[1]
    widthB = boxB[2] - boxB[0]
    heightB = boxB[3] - boxB[1]

    width_diff = abs(widthA - widthB) / widthA * 100  # Différence en pourcentage
    height_diff = abs(heightA - heightB) / heightA * 100  # Différence en pourcentage

    return width_diff, height_diff


In [5]:
def compute_mae(boxA, boxB):
    return np.mean(np.abs(np.array(boxA) - np.array(boxB)))

In [6]:
def compute_center(box):
    return ((box[0] + box[2]) / 2, (box[1] + box[3]) / 2)

In [ ]:
import os
import numpy as np
from PIL import Image, ImageEnhance

# Paramètres d'ajustement
contrast_factor = 1   # Augmenter le contraste (1.0 = original)
brightness_factor = 1 # Réduire la luminosité
num_occlusions = 0      # Nombre d'occlusions

CF = [0.25, 0.5, 0.75, 1, 1.5, 2, 3, 5]
BF = [0.25, 0.5, 0.75, 1, 1.5, 2, 3, 5]
OCCLUSIONS = [0, 1, 2, 3, 4, 5]

IMAGES = ['bag', 'bear', 'book', 'camel', 'rhino', 'swan']

for ig in IMAGES:
    for num_occlusions in OCCLUSIONS:
        # Dossier contenant les images
        input_folder = f'../../test/{ig}/'
        output_folder = f'../../test/processed/{ig}_occlusions_{num_occlusions}/'

        # Création du dossier de sortie
        os.makedirs(output_folder, exist_ok=True)

        def adjust_contrast(image, factor):
            """Ajuste le contraste de l'image. Factor > 1 augmente le contraste, factor < 1 le réduit."""
            enhancer = ImageEnhance.Contrast(image)
            return enhancer.enhance(factor)

        def adjust_brightness(image, factor):
            """Ajuste la luminosité de l'image. Factor > 1 augmente la luminosité, factor < 1 la réduit."""
            enhancer = ImageEnhance.Brightness(image)
            return enhancer.enhance(factor)

        def add_occlusion(image, num_rectangles=3):
            """Ajoute des occlusions en dessinant des rectangles noirs aléatoires."""
            img_array = np.array(image)
            h, w, _ = img_array.shape

            for _ in range(num_rectangles):
                x1, y1 = np.random.randint(0, w), np.random.randint(0, h)
                x2, y2 = np.random.randint(x1, w), np.random.randint(y1, h)
                img_array[y1:y2, x1:x2] = 0  # Rectangle noir

            return Image.fromarray(img_array)

        # Traitement des images
        for filename in os.listdir(input_folder):
            if filename.lower().endswith((".bmp")):
                img_path = os.path.join(input_folder, filename)
                image = Image.open(img_path).convert("RGB")

                # Appliquer les transformations
                image = adjust_contrast(image, contrast_factor)
                image = adjust_brightness(image, brightness_factor)
                image = add_occlusion(image, num_occlusions)

                # Sauvegarde
                output_path = os.path.join(output_folder, filename)
                image.save(output_path)
                print(f"Image traitée et enregistrée : {output_path}")

    print("Traitement terminé.")


Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-001.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-002.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-003.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-004.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-005.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-006.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-007.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-008.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-009.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-010.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-011.bmp
Image traitée et enregistrée : ../../test/processed/bag_occlusions_0/bag-012.bmp
Image traitée et enregistrée

In [9]:
all_results = []
ground_truth_files = [
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    '../data/bounding_box_bag.csv', 
    '../data/bounding_box_bear.csv',
    '../data/bounding_box_book.csv',
    '../data/bounding_box_camel.csv',
    '../data/bounding_box_rhino.csv',
    '../data/bounding_box_swan.csv',
    # Ajoute ici d'autres fichiers CSV ground truth
]

predicted_files = [
    '../data/predicted_bb_bag.csv', 
    '../data/predicted_bb_bear.csv',
    '../data/predicted_bb_book.csv',
    '../data/predicted_bb_camel.csv',
    '../data/predicted_bb_rhino.csv',
    '../data/predicted_bb_swan.csv',
        '../data/predicted_bb_bag_contrast_0.25.csv',
    '../data/predicted_bb_bear_contrast_0.25.csv',
    '../data/predicted_bb_book_contrast_0.25.csv',
    '../data/predicted_bb_camel_contrast_0.25.csv',
    '../data/predicted_bb_rhino_contrast_0.25.csv',
    '../data/predicted_bb_swan_contrast_0.25.csv',
    '../data/predicted_bb_bag_contrast_0.5.csv',
    '../data/predicted_bb_bear_contrast_0.5.csv',
    '../data/predicted_bb_book_contrast_0.5.csv',
    '../data/predicted_bb_camel_contrast_0.5.csv',
    '../data/predicted_bb_rhino_contrast_0.5.csv',
    '../data/predicted_bb_swan_contrast_0.5.csv',
    '../data/predicted_bb_bag_contrast_0.75.csv',
    '../data/predicted_bb_bear_contrast_0.75.csv',
    '../data/predicted_bb_book_contrast_0.75.csv',
    '../data/predicted_bb_camel_contrast_0.75.csv',
    '../data/predicted_bb_rhino_contrast_0.75.csv',
    '../data/predicted_bb_swan_contrast_0.75.csv',
    '../data/predicted_bb_bag_contrast_1.5.csv',
    '../data/predicted_bb_bear_contrast_1.5.csv',
    '../data/predicted_bb_book_contrast_1.5.csv',
    '../data/predicted_bb_camel_contrast_1.5.csv',
    '../data/predicted_bb_rhino_contrast_1.5.csv',
    '../data/predicted_bb_swan_contrast_1.5.csv',
    '../data/predicted_bb_bag_contrast_2.csv',
    '../data/predicted_bb_bear_contrast_2.csv',
    '../data/predicted_bb_book_contrast_2.csv',
    '../data/predicted_bb_camel_contrast_2.csv',
    '../data/predicted_bb_rhino_contrast_2.csv',
    '../data/predicted_bb_swan_contrast_2.csv',
    '../data/predicted_bb_bag_contrast_3.csv',
    '../data/predicted_bb_bear_contrast_3.csv',
    '../data/predicted_bb_book_contrast_3.csv',
    '../data/predicted_bb_camel_contrast_3.csv',
    '../data/predicted_bb_rhino_contrast_3.csv',
    '../data/predicted_bb_swan_contrast_3.csv',
    '../data/predicted_bb_bag_contrast_5.csv',
    '../data/predicted_bb_bear_contrast_5.csv',
    '../data/predicted_bb_book_contrast_5.csv',
    '../data/predicted_bb_camel_contrast_5.csv',
    '../data/predicted_bb_rhino_contrast_5.csv',
    '../data/predicted_bb_swan_contrast_5.csv',
    
    ]
# 📂 Parcourir chaque dossier de vidéo
for gt_file, pred_file in zip(ground_truth_files, predicted_files):
    # 📥 Charger les fichiers CSV
    gt_df = pd.read_csv(gt_file, header=None, names=["x1", "y1", "x2", "y2"])
    pred_df = pd.read_csv(pred_file, header=None, names=["x1", "y1", "x2", "y2"])

    if len(gt_df) != len(pred_df):
        print(" Nombre de frames différent pour un meme fichier")
        continue

    # 📊 Calcul des métriques
    ious = []
    distances = []
    width_diffs = []
    height_diffs = []

    for i in range(1,len(gt_df)):
        gt_first_row = gt_df.iloc[i]
        pred_first_row = pred_df.iloc[i]
        iou = compute_iou(
            [int(gt_first_row["x1"]), int(gt_first_row["y1"]), int(gt_first_row["x2"]), int(gt_first_row["y2"])],
            [int(pred_first_row["x1"]), int(pred_first_row["y1"]), int(pred_first_row["x2"]), int(pred_first_row["y2"])]
        )
        dist = compute_center_distance(
            [int(gt_first_row["x1"]), int(gt_first_row["y1"]), int(gt_first_row["x2"]), int(gt_first_row["y2"])],
            [int(pred_first_row["x1"]), int(pred_first_row["y1"]), int(pred_first_row["x2"]), int(pred_first_row["y2"])]
        )
        w_diff, h_diff = compute_size_difference(
            [int(gt_first_row["x1"]), int(gt_first_row["y1"]), int(gt_first_row["x2"]), int(gt_first_row["y2"])],
            [int(pred_first_row["x1"]), int(pred_first_row["y1"]), int(pred_first_row["x2"]), int(pred_first_row["y2"])]
        )

        ious.append(iou)
        distances.append(dist)
        width_diffs.append(w_diff)
        height_diffs.append(h_diff)

    # 📊 Résumé des métriques pour la vidéo
    avg_iou = np.mean(ious)
    avg_distance = np.mean(distances)
    avg_width_diff = np.mean(width_diffs)
    avg_height_diff = np.mean(height_diffs)

    all_results.append([gt_file, pred_file, avg_iou, avg_distance, avg_width_diff, avg_height_diff])

results_df = pd.DataFrame(all_results, columns=["Video File", "Prediction File", "Average IoU", "Average Distance", "Average Percentage Width Diff", "Average Percentage Height Diff"])
output_file = "./results_contrast.csv"
results_df.to_csv(output_file, index=False)

# 📊 Affichage des résultats sous forme de tableau
print(results_df)

                        Video File   
0     ../data/bounding_box_bag.csv  \
1    ../data/bounding_box_bear.csv   
2    ../data/bounding_box_book.csv   
3   ../data/bounding_box_camel.csv   
4   ../data/bounding_box_rhino.csv   
5    ../data/bounding_box_swan.csv   
6     ../data/bounding_box_bag.csv   
7    ../data/bounding_box_bear.csv   
8    ../data/bounding_box_book.csv   
9   ../data/bounding_box_camel.csv   
10  ../data/bounding_box_rhino.csv   
11   ../data/bounding_box_swan.csv   
12    ../data/bounding_box_bag.csv   
13   ../data/bounding_box_bear.csv   
14   ../data/bounding_box_book.csv   
15  ../data/bounding_box_camel.csv   
16  ../data/bounding_box_rhino.csv   
17   ../data/bounding_box_swan.csv   
18    ../data/bounding_box_bag.csv   
19   ../data/bounding_box_bear.csv   
20   ../data/bounding_box_book.csv   
21  ../data/bounding_box_camel.csv   
22  ../data/bounding_box_rhino.csv   
23   ../data/bounding_box_swan.csv   
24    ../data/bounding_box_bag.csv   
25   ../data

In [ ]:
def create_tracking_video(image_folder, gt_csv, pred_csv, output_video_path, frame_rate=30, video_size=(640, 480)):
    """
    Crée une vidéo avec le vrai tracking (ground truth) et le tracking prédit.
    
    @param image_folder: Le dossier contenant les images (frames) de la vidéo.
    @param gt_csv: Le fichier CSV contenant les boîtes de ground truth.
    @param pred_csv: Le fichier CSV contenant les boîtes prédites.
    @param output_video_path: Le chemin où enregistrer la vidéo générée.
    @param frame_rate: Le taux de rafraîchissement de la vidéo (en images par seconde).
    @param video_size: La taille de la vidéo (largeur, hauteur).
    """
    
    # Charger les données ground truth et predicted
    gt_df = pd.read_csv(gt_csv, header=None, names=["x1", "y1", "x2", "y2"])
    pred_df = pd.read_csv(pred_csv, header=None, names=["x1", "y1", "x2", "y2"])

    # S'assurer que le nombre de frames est identique dans les deux fichiers CSV
    if len(gt_df) != len(pred_df):
        print("Le nombre de frames ne correspond pas entre les fichiers ground truth et predicted.")
        return

    # Liste des fichiers d'image triée
    image_files = sorted(os.listdir(image_folder))
    
    # Initialiser la vidéo de sortie (en format AVI avec codec MJPEG)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Codec MJPEG utilisé pour les vidéos AVI
    out_video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, video_size)

    # Pour chaque frame, dessiner les boîtes de ground truth et prédictions
    for i, image_file in enumerate(image_files):
        if not image_file.endswith(".bmp"):  # Vérifier si l'image est bien au format .bmp
            continue
        
        # Charger l'image
        image_path = os.path.join(image_folder, image_file)
        frame = cv2.imread(image_path)

        # Vérifier que l'image a bien été chargée
        if frame is None:
            print(f"Erreur lors du chargement de l'image {image_file}.")
            continue

        # Obtenir les dimensions d'origine de l'image
        original_height, original_width = frame.shape[:2]

        # Redimensionner l'image pour qu'elle corresponde à la taille de la vidéo
        frame_resized = cv2.resize(frame, video_size)
        resized_height, resized_width = frame_resized.shape[:2]

        # Calculer le facteur de redimensionnement
        scale_x = resized_width / original_width
        scale_y = resized_height / original_height

        # Récupérer les coordonnées des boxes ground truth et predicted pour cette frame
        gt_box = gt_df.iloc[i]
        pred_box = pred_df.iloc[i]

        # Appliquer le redimensionnement aux coordonnées des boîtes
        gt_x1, gt_y1, gt_x2, gt_y2 = (
            int(gt_box["x1"] * scale_x),
            int(gt_box["y1"] * scale_y),
            int(gt_box["x2"] * scale_x),
            int(gt_box["y2"] * scale_y),
        )
        pred_x1, pred_y1, pred_x2, pred_y2 = (
            int(pred_box["x1"] * scale_x),
            int(pred_box["y1"] * scale_y),
            int(pred_box["x2"] * scale_x),
            int(pred_box["y2"] * scale_y),
        )

        # Dessiner les boîtes de ground truth (en vert)
        cv2.rectangle(frame_resized, (gt_x1, gt_y1), (gt_x2, gt_y2), (0, 255, 0), 2)
        cv2.putText(frame_resized, "Ground Truth", (gt_x1, gt_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Dessiner les boîtes prédites (en rouge)
        cv2.rectangle(frame_resized, (pred_x1, pred_y1), (pred_x2, pred_y2), (0, 0, 255), 2)
        cv2.putText(frame_resized, "Predicted", (pred_x1, pred_y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Ajouter l'image au fichier vidéo de sortie
        out_video.write(frame_resized)

    # Libérer les ressources vidéo
    out_video.release()
    print(f"Vidéo de tracking générée : {output_video_path}")

# Exemple d'utilisation :
image_folder = '../../test/rhino'  # Dossier des images (frames de la vidéo)
gt_csv = 'ground_truth/bounding_box_rhino.csv'  # Fichier CSV des ground truth
pred_csv = 'predicted/bounding_box_rhino_predicted.csv'  # Fichier CSV des prédictions
output_video_path = 'rhino_video.avi'  # Chemin du fichier vidéo de sortie en AVI

create_tracking_video(image_folder, gt_csv, pred_csv, output_video_path)


Vidéo de tracking générée : rhino_video.avi


In [ ]:
# Generate centroid file for each video


predicted_files = [
    '../data/bounding_box_cow_predicted.csv',
    '../data/bounding_box_fish_predicted.csv',
    '../data/bounding_box_octopus_predicted.csv',
]

img = {'../data/bounding_box_cow_predicted.csv' : 'cow', '../data/bounding_box_fish_predicted.csv' : 'fish', '../data/bounding_box_octopus_predicted.csv' : 'octopus'}

for pred_file in predicted_files:
    # Charger le fichier CSV
    pred_df = pd.read_csv(pred_file, header=None, names=["x1", "y1", "x2", "y2"])

    # Créer un DataFrame pour les coordonnées des centroids
    centroids = []

    # Calculer les coordonnées des centroids pour chaque frame
    for i in range(2,len(pred_df)):
        pred_row = pred_df.iloc[i]
        x = (int(pred_row["x1"]) + int(pred_row["x2"])) / 2
        y = (int(pred_row["y1"]) + int(pred_row["y2"])) / 2
        centroids.append([x, y])
    
    # Sauvegarder les coordonnées des centroids dans un fichier CSV
    output_path = f'./centroids_{img[pred_file]}.npy'
    output_file = np.save(output_path, centroids)